## Ej1: Interpretación de un modelo de red neuronal

- ¿Qué tipo de aprendizaje estamos utilizando? ¿Qué tipo de problema se busca resolver?
- Explica la arquitectura de la red neuronal utilizada y los hiperparámetros utilizados.
- Explica el proceso de entrenamiento y evaluación. ¿Está funcionando bien el modelo?
- ¿Qué mejoras se podrían proponer para un mejor abordaje del problema?

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
from torch.nn import functional as F
import torch.optim as optim

_ = torch.manual_seed(42)

df = pd.read_csv("data/dataset.csv")

X = df.drop('value', axis=1)
y = df['value']

X_train_np, X_test_np, y_train_np, y_test_np = train_test_split(X, y, test_size=0.2)

X_train = torch.FloatTensor(X_train_np.values)
y_train = torch.FloatTensor(y_train_np.values).view(-1, 1)
X_test = torch.FloatTensor(X_test_np.values)
y_test = torch.FloatTensor(y_test_np.values)

class Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(30, 128)
        self.fc2 = nn.Linear(128, 128)
        self.fc3 = nn.Linear(128, 1)
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.sigmoid(self.fc3(x))
        return x    

model = Model()

criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=5e-4)

num_epochs = 7
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()
    
    outputs = model(X_train)
    loss = criterion(outputs, y_train)
    
    loss.backward()
    optimizer.step()
    
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item()}')
    
model.eval()
with torch.no_grad():
    outputs = model(X_test)
    _, predicted = torch.max(outputs, 1)
    accuracy = (predicted == y_test).sum().item() / len(y_test)
    print(f'Accuracy on the test set: {accuracy}')


Epoch [1/7], Loss: 99.59390258789062
Epoch [2/7], Loss: 99.44441986083984
Epoch [3/7], Loss: 98.86461639404297
Epoch [4/7], Loss: 0.21135300397872925
Epoch [5/7], Loss: 0.17388787865638733
Epoch [6/7], Loss: 0.1732185035943985
Epoch [7/7], Loss: 0.17313863337039948
Accuracy on the test set: 0.9982971103542713


In [2]:
from sklearn.metrics import classification_report, confusion_matrix
print(classification_report(y_test, predicted))
print(confusion_matrix(y_test, predicted))

y.value_counts()

              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00     56865
         1.0       0.00      0.00      0.00        97

    accuracy                           1.00     56962
   macro avg       0.50      0.50      0.50     56962
weighted avg       1.00      1.00      1.00     56962

[[56865     0]
 [   97     0]]


/home/avidaldo/miniconda3/envs/env_torch/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/avidaldo/miniconda3/envs/env_torch/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/avidaldo/miniconda3/envs/env_torch/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f

value
0    284315
1       492
Name: count, dtype: int64

### Solución

Dataset original: https://www.kaggle.com/datasets/mlg-ulb/creditcardfraud
(modificados nombres de últimas dos columnas y fichero)

Se trata de un problema de clasificación binaria (aprendizaje supervisado), con la columna "value" como variable objetivo con dos posibles clases ya codificadas con los valores "0" y "1" y otras 30 *features* predictoras.

Se utiliza una red neuronal con tres capas densas (*fully connected*): una de entrada con 30 neuronas, una oculta de 128 y una de salidad de una única neurona. La entrada tiene que ser de tamaño 30 correspondientes con cada una de las 30 *features*. La función de activación utilizada es ReLU en las dos primeras capas. Solo es necesario una neurona de salida ya que es un problema de clasificación binaria. La salida es una única neurona activada con una función sigmoide, devolviendo de este modo la probabilidad de que la instancia pertenezca a la clase "1".

El modelo se entrena durante 7 *epochs* sin *batch normalization* y se evalúa con el conjunto de test. Se observa un salto llamativo en la función de pérdida tras la 3ª *epoch*. La *accuracy* obtenida es del 99,82%, extremadamente alta. Haciendo modificaciones simples en el modelo o simplemente haciendo pruebas con menos epochs, se puede observar que la precisión final se mantiene en esos números. El modelo no parece estar aprendiendo sino que consigue esa precisión desde el principio.

Mostrando un *classification report* y/o la matriz de confusión se observa que el modelo no es capaz de detectar ninguna de las instancias de la clase minoritaria. Esto se debe a que las dos clases del dataset no están equilibradas (*imbalanced dataset*), lo que podemos confirmar mostrando la distribución de las clases.

Por tanto para abordar correctamente el problema será necesario definir otras métricas de evaluación, como la precisión, el *recall* o el *F1-score*, y utilizar técnicas de balanceo de clases, como *oversampling* o *undersampling*.
